In [1]:
import pandas as pd
from tqdm import tqdm
import re

# 테스트 데이터셋 준비

In [2]:
test_df = pd.read_csv("/opt/ml/input/data/test_data.csv")
test_df.keys()

Index(['id', 'db', 'prod_id', 'prod_name', 'context', 'filtered_context',
       'summary', 'filter_score', 'final_summary'],
      dtype='object')

In [3]:
test_dataset = []

for idx, item in test_df.iterrows():
    data = {
        "id": item["id"],
        "prod_name": item["prod_name"],
        "review": item["filtered_context"],
        "summary": item["final_summary"]
    }
    test_dataset.append(data)
len(test_dataset)

50

# 평가할 결과 데이터 준비

In [4]:
RES_PATH = "/opt/ml/input/output/outout_T5_g256.csv"
result_df = pd.read_csv(RES_PATH)
result_df.head(1)

,Unnamed: 0,id,output,time
0,0,1,<가격> 가격도 착한데 쌀 품질도 좋아요 <맛> 맛이 좋고 윤기도 흘러요 <맛> 밥...,"['0:00:07', '0:00:07', '0:00:07', '0:00:07', '..."


In [5]:
# 컬럼명
ID = "id"
# OUTPUT = "predicted_summary" 
OUTPUT = "output"
# OUTPUT = "preds"
TIME = "time"
# TIME = "times"

In [7]:
from utils.preprocess import clean_text

def string2output(t):
    t = t[1:-1]
    t = [
        clean_text(tt[1:-1], remove_tag=False)
        for tt in t.split(",")
        ]
    
    return " ".join(t)

text = result_df.loc[0, OUTPUT]
print(string2output(text))

격> 가격도 착한데 쌀 품질도 좋아요 <맛> 맛이 좋고 윤기도 흘러요 <맛> 밥할 때마다 구수한 냄새가 입맛이 확 돋아나게 합니다 <향> 개봉 시 구수한 백미 냄새가 나며 알이 고르고 깨끗합니다 <향> 취사 시 특유의 백미 냄새가 두루 퍼지고 달달하면서 고소한 맛이 나는 쌀입니다 <맛> 2인 맞벌이 가구라서 평소 집에서 밥해 먹을 일이 많이 없어서 큰 쌀 사기 부담스러운데 4kg 짜리가 아주 좋네요 <품질> 2022년도 쌀이고 도정 일은 아주 최근 일이라서 품질이 좋네요 <맛> 쌀알도 매끈하고 윤기도 있어서 흠잡을 건 따로 없고 맛은 물론 깔끔하고 담백한 느낌입니다 <맛> 저희 집 입맛에는 아주 만족스러운 맛이에요 <맛> 곰곰 소중한 우리 <선호> 탄수화물이 풍부 <맛> 곰곰 쌀로 밥하니까 밥이 고슬고슬 맛있게 돼서 밥을 많이 먹게 되는 게 함정 <맛> 쌀알 굵기도 적당하고 해서 밥했을 때 더욱 맛나나봐요 <맛> 가한 밥이 윤기가 좔좔 나는 게 곰곰 쌀 안 고를 이유가 없죠 <맛> 10kg 면 제법 오래 먹더라고요 <맛> 찰기가 엄청나고 윤기가 흐르지는 않지만 평타 치는 백미 맛이었어요 <선호> 쌀을 이렇게 사는 건 또 처음입니다 <선호> 쌀을 이렇게 사는 건 또 처음입니다 <선호> 쌀을 이렇게 사는 건 또 처음입니다 <선호> 쌀을 이렇게 사는 건 또 처음입니다 <선호> 쌀을 이렇게 사는 건 또 처음입니다 <선호> 쌀을 이렇게 사는 건 또 처음입니다 <선호 <선호> 2kg 용량 강추 <가성비> 용량이 큰 걸 살수록 저렴 <맛> 건강한 맛과 편의함을 제공 <선호> 곰곰 제품이 참 많네요 <선호> 판매량이 높으니 계속해서 최근 도정일자로 공급되고 판매되고의 반복 <맛> 다른 잡곡 안 섞고 이 쌀만 해먹어도 찰기가 있다는 느낌 <맛> 어떤 쌀은 밥을 하면 푸석거리는데 이건 그런 것 없네요 <품질> 품질도 우수하고 괜찮았어요 <품질> 질 좋을 품질을 착한 가격에 먹을 수 있어서 소비자 입장에서는 너무 좋은 것 같아요 <품질> 암 도정일이 2023년 5월 22일 날짜가

In [8]:
def to_sec(t):
    hr, mint, sec = [int(i) for i in t.split(":")]
    sec += mint*60 + hr*60*60
    
    return sec

In [9]:
def string2sec(t):
    t = t[1:-1].split(",")
    t = [
        to_sec(tt.strip()[1:-1])
        for tt in t
    ]
    
    return t
    
t = result_df.loc[7, TIME]
t = string2sec(t)
print(t, sum(t))

[7, 7, 7, 7, 7, 6] 41


In [10]:
def float_string2sec(t):
    t = t[1:-1]
    t = [
        float(tt)
        for tt in t.split(",")
        ]
    
    return t

print(float_string2sec(result_df.loc[0, TIME]))

ValueError: could not convert string to float: "'0:00:07'"

In [11]:
from utils.preprocess import clean_text

preds, times = [], []

for idx, item in result_df.iterrows():
    output = clean_text(item[OUTPUT], remove_tag=False)
    # output = string2output(item[OUTPUT])
    preds.append(output)
    
    if TIME not in result_df.keys():
        times.append(-1)
    else:
        time_list = string2sec(item[TIME])
        # time_list = float_string2sec(item[TIME])
        times.append(sum(time_list)) 

preds[0][:20], times[0]

('<가격> 가격도 착한데 쌀 품질도 좋', 35)

## 파일에 저장

In [12]:
import json
import os

d = {
    "test_dataset": test_dataset,
    "preds": preds,
    "times": times,
    "path": RES_PATH
}

filename, _ = os.path.splitext(RES_PATH)
READY_PATH = filename + "_ready.json"

print(READY_PATH)

with open(READY_PATH, "w", encoding="utf-8") as f:
    json.dump(d, f, ensure_ascii=False)

/opt/ml/input/output/outout_T5_g256_ready.json


# 평가하기

In [ ]:
from transformers import AutoModel, AutoTokenizer

SENT_MODEL = "BM-K/KoSimCSE-roberta"
sentence_model = AutoModel.from_pretrained(SENT_MODEL)
sentence_tokenizer = AutoTokenizer.from_pretrained(SENT_MODEL)

In [75]:
import os

from summary_evaluate import evaluate
from summary_utils import save_evaluation

dirname, filename = os.path.split(RES_PATH)
filename, _ = os.path.splitext(filename)
filename = filename + "_scores"

eval_result = evaluate(
    preds, test_dataset, sentence_model, sentence_tokenizer, times
)


save_evaluation(
    eval_result,
    dir_name=dirname, name=filename
    )

Evaluating: 100%|██████████| 50/50 [12:32<00:00, 15.05s/it]

File saved at:  /opt/ml/input/output/outout_T5_base4_scores_07260250.json


## 전체 길이

In [13]:
import json

FILES = [
    "/opt/ml/input/output/outout_T5_g256_scores_07280017.json"    
]

length_score = []

for path in FILES:
    with open(path, "r") as f:
        evaluation = json.load(f)
        
    length_diff = []
    for item in evaluation["results"]:
        length_diff.append(item["length_diff"])
    length_score.append(sum(length_diff) / len(length_diff))

    print(path, length_score[-1])

/opt/ml/input/output/outout_T5_g256_scores_07280017.json 177.5


In [2]:
print(length_diff)

[239, 251, 201, 152, 250, 282, 222, 241, 321, 235, 205, 202, 344, 160, 215, 152, 185, 156, 126, 102, 308, 173, 241, 160, 188, 155, 117, 138, 126, 156, 184, 256, 146, 154, 126, 165, 213, 186, 254, 94, 214, 103, 77, 85, 68, 104, 59, 125, 265, 139]
